In [1]:
#可以手动打乱原始数据集，直接在excel表中操作
import os
import csv
import random

#数据集地址
data_dir = '../input/petfinder-pawpularity-score'
origin_dir = os.path.join(data_dir,'train.csv')
train_dir = './train_data.csv'

train_file = open(train_dir,'w',newline='')
#读取源数据地址，在train.csv文件夹下
with open(origin_dir,'r') as f:
    lines = f.readlines()[1:] #  去掉跳过文件头行
f.close()
#打乱读入的每一行数据（样例顺序打乱）
random.shuffle(lines)
#写文件
train_writer = csv.writer(train_file)

for i in range(len(lines)):
    temp = lines[i].rsplit('\n')[0].split(',')
    # 取大约80%的样例为训练集，剩余的为测试集
    if i <= 2000:
        train_writer.writerow(temp)

train_file.close()

In [2]:
import torch
import torch.nn as nn
try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url


__all__ = [
    'VGG', 'vgg16',
]


model_urls = {
    'vgg16': 'https://download.pytorch.org/models/vgg16-397923af.pth',
}


class VGG(nn.Module):

    def __init__(self, features, num_classes=1000, init_weights=True):
        super(VGG, self).__init__()
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        ##将VGG16的特征提取层参数进行冻结，不对其进行更新
        for param in self.features.parameters():
            param.requires_grad_(False)
        self.Line = nn.Sequential(
            nn.Linear(512 * 7 * 7, 512),
            nn.ReLU(True),
            nn.Linear(512, 256)
        )
        self.model2 = nn.Sequential(
            nn.Linear(12, 128),
            nn.ReLU(True),
            nn.Linear(128, 256)
        )
        self.classifier = nn.Sequential(  # 定义自己的分类层
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.Linear(256, 100)
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, input1, input2):
        s = input1.size(0)
        input1 = self.features(input1)
        input1 = input1.view(s, -1)
        input1 = self.Line(input1)
        input1 = input1.view(s, -1)

        input2 = input2.view(s, -1)
        input2 = self.model2(input2)
        input2 = input2.view(s, -1)

        Input = torch.cat([input1, input2], dim=1)
        output = self.classifier(Input)
        return output

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfgs = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


def _vgg(arch, cfg, batch_norm, pretrained, progress, **kwargs):
    if pretrained:
        kwargs['init_weights'] = False
    model = VGG(make_layers(cfgs[cfg], batch_norm=batch_norm), **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model


def vgg16(pretrained=False, progress=True, **kwargs):
    r"""VGG 16-layer model (configuration "D")
    `"Very Deep Convolutional Networks For Large-Scale Image Recognition" <https://arxiv.org/pdf/1409.1556.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _vgg('vgg16', 'D', False, pretrained, progress, **kwargs)

from torch.utils.data import Dataset
from PIL import Image
import os
import torch
import numpy as np
from torchvision import transforms
import torchvision
from torch import nn
from torch.utils.data import DataLoader
import time
import matplotlib.pyplot as plt
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

class MyData(Dataset):

    def __init__(self, root_dir, image_dir, label_dir, transform=None):
        self.root_dir = root_dir
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.label_path = os.path.join(self.root_dir, self.label_dir)
        self.image_path = os.path.join(self.root_dir, self.image_dir)
        self.image_list = os.listdir(self.image_path)
        self.transform = transform

        with open(self.label_path, 'r') as f:
            self.data = f.readlines()[1:]  # 去掉跳过文件头行
        f.close()

    def __getitem__(self, idx):
        img_name = self.data[idx].rsplit('\n')[0].split(',')[0] + '.jpg'
        img_item_path = os.path.join(self.root_dir, self.image_dir, img_name)
        img = Image.open(img_item_path)
        feature = [float(x) for x in self.data[idx].rsplit('\n')[0].split(',')[1:13]]
        target = int(self.data[idx].rsplit('\n')[0].split(',')[13])-1
        img = self.transform(img)
        # trans_norm = transforms.Normalize([0.5, 0.5, 0.5])
        # trans_norm1 = transforms.Normalize([0.5,])
        # img = trans_norm(img)
        feature = torch.as_tensor(feature)
        feature = torch.reshape(feature, (1, 1, -1))
        # feature = trans_norm1(feature)
        label = torch.as_tensor(target)
        return img, feature, label

    def __len__(self):
        return len(self.data)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
# 准备数据集
# 这里把PetFInder作为根目录，右击该文件夹->Make Directory as...->root
root_dir = "/kaggle/input/petfinder-pawpularity-score"
image1_dir = "train"
label1_dir = "./train_data.csv"
dataset_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),  # 随机长宽比裁剪为224*224
    transforms.RandomHorizontalFlip(),  # 依概率p=0.5水平翻转
    transforms.ToTensor(),  # 转化为张量并归一化为[0-1]
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
train_dataset = MyData("", os.path.join(root_dir, image1_dir), label1_dir, transform=dataset_transform)

# length 长度
train_data_size = len(train_dataset)
# 如果train_data_size=10, 训练数据集的长度为：10
print("训练数据集的长度为：{}".format(train_data_size))

# 格式转换
# 利用 DataLoader 来加载数据集
batch = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch, drop_last=False)
# 创建网络模型
model = vgg16(pretrained=False)
# 第一次运行时将下面导入权值的代码注释掉
#model.load_state_dict(torch.load('./VGG16_Cats_Dogs_loss.pth'))
model = model.to(device)

# 损失函数
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)
optimizer = torch.optim.SGD(model.classifier.parameters(), lr=0.01, momentum=0.5)

# 保存每个epoch后的Accuracy Loss Val_Accuracy
Accuracy = []
Loss = []
Val_Accuracy = []
BEST_VAL_ACC = 0.
BEST_TR_ACC = 0.
Min_Loss = 100.
# 第一次运行时将下面导入当前最小损失值的代码注释掉
#Min_Loss = np.loadtxt('./vgg_loss_acc.txt')
# 训练

for epoch in range(5):
    print("---------------------第 {} 轮训练开始---------------------".format(epoch + 1))
    since = time.time()
    total_train_loss = 0.
    total_accuracy = 0.
    model.train()
    for i,(imgs, feature, labels) in enumerate(train_dataloader, 0):
        imgs = imgs.to(device)
        feature = feature.to(device)
        labels = labels.to(device)

        # 优化器优化模型
        optimizer.zero_grad()
        outs = model(imgs, feature)
        loss = loss_fn(outs, labels)
        loss.backward()
        optimizer.step()

        #      输出状态
        total_train_loss = total_train_loss + loss.item()
        _, prediction = torch.max(outs, 1)
        total_accuracy += (prediction == labels).sum().item()

        if i % 20 == 19:
            print('epoch {}, iter{}, loss: {:.4f}'.format(epoch+1,i+1, loss.item() / batch))
    Loss.append(total_train_loss / train_data_size)
    Accuracy.append(total_accuracy / train_data_size)
    print("epoch {}在整体训练集上的平均损失: {:.4f}".format(epoch+1,total_train_loss / train_data_size))
    print("epoch {}在整体训练集上的正确率: {:.4f}%".format(epoch+1,100*total_accuracy / train_data_size))
    if Loss[epoch] < Min_Loss:
        print('Find Better Model and Saving it...')
        torch.save(model.state_dict(), './VGG16_Cats_Dogs_loss.pth')
        Min_Loss = Loss[epoch]
        ACC = []
        ACC.append(Min_Loss)
        np.savetxt('./vgg_loss_acc.txt',ACC,fmt='%.04f' )
        print('Saved!')
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Now the minimum loss is {:.4f}'.format(Min_Loss))

训练数据集的长度为：2000
---------------------第 1 轮训练开始---------------------
epoch 1, iter20, loss: 0.0719
epoch 1在整体训练集上的平均损失: 0.0736
epoch 1在整体训练集上的正确率: 3.1500%
Find Better Model and Saving it...
Saved!
Training complete in 0m 50s
Now the minimum loss is 0.0736
---------------------第 2 轮训练开始---------------------
epoch 2, iter20, loss: 0.0718
epoch 2在整体训练集上的平均损失: 0.0735
epoch 2在整体训练集上的正确率: 2.9500%
Find Better Model and Saving it...
Saved!
Training complete in 0m 35s
Now the minimum loss is 0.0735
---------------------第 3 轮训练开始---------------------
epoch 3, iter20, loss: 0.0717
epoch 3在整体训练集上的平均损失: 0.0735
epoch 3在整体训练集上的正确率: 3.2000%
Find Better Model and Saving it...
Saved!
Training complete in 0m 36s
Now the minimum loss is 0.0735
---------------------第 4 轮训练开始---------------------
epoch 4, iter20, loss: 0.0716
epoch 4在整体训练集上的平均损失: 0.0734
epoch 4在整体训练集上的正确率: 3.4000%
Find Better Model and Saving it...
Saved!
Training complete in 0m 36s
Now the minimum loss is 0.0734
---------------------第 5 轮训练开

In [3]:
from torch.utils.data import Dataset
from PIL import Image
import os
import torch
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader
import pandas as pd
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
class TestData(Dataset):

    def __init__(self, root_dir, image_dir, label_dir, transform=None):
        self.root_dir = root_dir
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.label_path = os.path.join(self.root_dir, self.label_dir)
        self.image_path = os.path.join(self.root_dir, self.image_dir)
        self.image_list = os.listdir(self.image_path)
        self.transform = transform

        with open(self.label_path, 'r') as f:
            self.data = f.readlines()[1:]  # 去掉跳过文件头行
        f.close()

    def __getitem__(self, idx):
        img_name = self.data[idx].rsplit('\n')[0].split(',')[0] + '.jpg'
        img_item_path = os.path.join(self.root_dir, self.image_dir, img_name)
        img = Image.open(img_item_path)
        feature = [float(x) for x in self.data[idx].rsplit('\n')[0].split(',')[1:13]]
        img = self.transform(img)
        feature = torch.as_tensor(feature)
        feature = torch.reshape(feature, (1, 1, -1))
        return img, feature

    def __len__(self):
        return len(self.data)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#准备数据集
root_dir = "/kaggle/input/petfinder-pawpularity-score"
label3_dir = "test.csv"
dataset_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),  # 随机长宽比裁剪为224*224
    transforms.RandomHorizontalFlip(),  # 依概率p=0.5水平翻转
    transforms.ToTensor(),  # 转化为张量并归一化为[0-1]
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test = TestData(root_dir,"test",label3_dir,transform=dataset_transform)
test_dataloader = DataLoader(test, batch_size=8, drop_last=False)

# 创建网络模型
model = vgg16(pretrained=False)
# 第一次运行时将下面导入权值的代码注释掉
model.load_state_dict(torch.load('./VGG16_Cats_Dogs_loss.pth'))
model = model.to(device)
# 测试
pred_list = []
with open(os.path.join(root_dir, 'sample_submission.csv'), 'r') as f:
    test_label = f.readlines()[1:] # 去掉跳过文件头行
f.close()

model.eval()
with torch.no_grad():
    for data in test_dataloader:
        imgs, feature = data
        imgs = imgs.to(device)
        feature = feature.to(device)
        out = model(imgs, feature)
        _, prediction = torch.max(out, 1)
id_list = []
targets = []
for i in range(len(test_label)):
    id_list.append(test_label[i].rsplit('\n')[0].split(',')[0])
    targets.append(float(test_label[i].rsplit('\n')[0].split(',')[1]))
pred_list = prediction.cuda().data.cpu().numpy()
pred_list = list(pred_list)
for i in range(len(pred_list)):
    pred_list[i] = float('%.02f' % (float(int(pred_list[i]))+0.01))
print(targets)
print(pred_list)
res = pd.DataFrame({
    'Id': id_list,
    'Pawpularity': pred_list
})
res.to_csv('./submission.csv', index=False)

[67.75, 59.15, 20.02, 94.53, 89.82, 65.5, 71.42, 5.85]
[27.01, 27.01, 27.01, 27.01, 27.01, 27.01, 27.01, 27.01]
